In [1]:
load_ext Cython

In [7]:
players = 2
def hash_transform(state):
    h = ''
    h +='{:02d}'.format(state['players'])
    h +='{:02d}'.format(len(state['history']))
    h += '{:02}'.format(state['card'].score)
    if state['passed']:
        h += '{:02}'.format(state['passed'].score)
    else:
        h += '00'
    for item in state['history']:
        if item == 'pass' or item == 'cut':
            h += 'p'
        else:
            h += 's'
    return h

reverse_value_dict = {'A':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'10':10,'J':11,'Q':12,'K':13}
class Card(object):
    def __init__(self,value='A',suit='♠'):
        self.value = value
        self.suit = suit
        self.score = reverse_value_dict[value]
    def __str__(self):
        return "{}{}".format(self.value,self.suit)
    def __repr__(self):
        return str(self)
import sys
def progress(p,size = 20):
    p = min(p,0.9999)
    filled = int(p*size)
    sys.stdout.write('[' + '#'*filled + ' '*(size-filled) + ']' + '{:6.2f}% complete\r'.format(100.*p))
    
frozen = {}


In [12]:
%%cython -a
#cython: cdivision = True
#cython: profile=True
import itertools
from copy import copy
passing = {}
staying = {}
policy = {}
cpdef double get_score(list hands,int player):
    cdef int ahands[12]
    cdef int nhands
    cdef int i
    cdef int smallest
    cdef double score[12]
    cdef double normsumscore=0
    cdef double increment
    
    nhands = len(hands)
    smallest = 14
    for i in xrange(nhands):
        ahands[i] = hands[i]
        if ahands[i] < smallest:
            smallest = ahands[i]
    
    increment = <double> 1 / nhands
    for i in xrange(nhands):
        if ahands[i] == smallest:
            score[i] = - 1
            normsumscore += increment
        else:
            score[i] = 0
    
    return score[player]+normsumscore


In [16]:


from copy import copy   
import re
INF = float('inf')
full_deck = {1:4 , 2:4, 3:4, 4:4, 5:4, 6:4, 7:4, 8:4, 9:4, 10:4, 11:4, 12:4, 13:4}
def key_gen(players = 2):
    for i in xrange(players):
        for j in xrange(1,14):
            for actions_list in itertools.product(['s','p','b','k'],repeat=i):
                actions = ''.join(actions_list)
                if re.search(r'[^b]k|b[^k]|^k|b$', actions): #don't bother generating impossible king-blocks
                    continue
                if not actions or actions[-1] != 'p':
                    yield "{:02d}{:02d}{:02d}{:02d}{}".format(players,i,j,0,actions)
                else:
                    for k in xrange(1,13): #can't pass king back
                        yield "{:02d}{:02d}{:02d}{:02d}{}".format(players,i,j,k,actions)                        
# for players in xrange(6):
#     for key in key_gen(players):
#         passing[key] = 1
#         staying[key] = 0  
deck = {1:4 , 2:4, 3:4, 4:4, 5:4, 6:4, 7:4, 8:4, 9:4, 10:4, 11:4, 12:4, 13:4}
def generate_hands(hands):
    global deck
    rel = 1
    if all(hands):
        yield rel
        raise StopIteration
    index = next(i for i in xrange(len(hands)) if not hands[i])
    assert(not index is None)
    for card in xrange(1,14):
        if deck[card] == 0:
            continue
        hands[index] = card
        rel = deck[card]
        deck[card] -= 1        
        for cumrel in generate_hands(hands):
            yield rel * cumrel
        deck[card] += 1
    hands[index] = None
                

def simulate(hands,key):
    #assumes that it is player's turn to act
    hands = copy(hands)
    players = int(key[:2])
    assert(len(hands) == players)
    player = int(key[2:4])
    original_player=player
    while player < players - 1:
        if policy[key][0] == 'p': #attempt to pass
            if player < players - 1 and hands[player + 1] == 13: #check for king
                action = 'bk'
            else:
                action = 'p'
                hands[player],hands[player + 1]=hands[player + 1],hands[player]
                       
        else:
            action = 's'
            
        player += len(action)
#         print players,player,hands,key,action
        if player == players:
            break
        key = "{:02d}{:02d}{:02d}{:02d}{}".format(
            players,player,hands[player],0 if action!='p' else hands[player-1],key[8:]+action)
        
    action = 'k' if len(key) == 8 + players else 's' if policy[key][0] != 'p' else 'p'
    if action == 's':
        return get_score(hands,original_player)
    else:
        global deck
        cumrel = 0.
        score = 0
        original_card = hands[-1]
        for cut in xrange(1,14):
            rel = deck[cut]
            hands[-1] = cut
            if rel > 0:
                score += rel*get_score(hands,original_player)
                cumrel += rel
        if cumrel > 0:
            score /= float(cumrel)
        hands[-1] = original_card
        return score
    

def allowed(hands,key): 
    
    players = int(key[:2])
    player = int(key[2:4])
    assert(int(key[:2]) == len(hands)) #hands is 1 larger than the number of players because it includes the cut card
    assert(int(key[4:6]) == hands[player])
    assert(len(key) == 8 + player)
    if int(key[6:8]):
        assert(int(key[6:8]) == hands[player-1])
    #reconstruct old hands
    handscopy = copy(hands)
    actions = key[8:] 
    index = 0
    for action in actions[::-1]:
        if action == 'p':
              handscopy[player - index],handscopy[player-index-1] = handscopy[player-index-1],handscopy[player-index]
        index += 1
    index = 0
    last_action = 's'
    true_actions = ''
    for action in actions:
#         if len(true_actions) > index:
#             index += 1
#             continue
        if last_action == 'k':
            index += 1
            continue
        key = "{:02d}{:02d}{:02d}{:02d}{}".format(
            players,index,handscopy[index],0 if last_action!='p' else handscopy[index-1],actions[:index])
#         print key,actions
        if not key in policy:
#             print 'not in policy'
            return False
        if policy[key][-1] == 'n':
#             print 'not allowed'
            return False
        if policy[key][0] == 'p': #attempt to pass
            if index < players - 1 and hands[index + 1] == 13: #check for king
                if actions[index] != 'b':
#                     print 'didnt block'
                    return False
                last_action = 'k'
#                 true_actions += 'bk'
            else:
                if actions[index] != 'p':
#                     print 'didnt pass'
                    return False
                last_action = 'p'
                handscopy[index],handscopy[index+1] = handscopy[index+1],handscopy[index]
#                 true_actions += 'p'
        else:
            if actions[index] != 's':
#                 print 'didnt stay'
                return False
            last_action = 's'
#             true_actions += 's'
#         last_action = true_actions[-1]
        index += 1
#         if not true_actions in actions:
#             return False
        
#     print hands , true_actions
#     return true_actions == actions
    return True





def equity(key):
    global deck
    deck = {1:4 , 2:4, 3:4, 4:4, 5:4, 6:4, 7:4, 8:4, 9:4, 10:4, 11:4, 12:4, 13:4}
    players = int(key[:2])
    player = int(key[2:4])
    hands = [0]*(players)
    hands[player] = int(key[4:6])
    deck[hands[player]] -= 1
    if key[-1] == 'p':
        hands[player-1] = int(key[6:8])
        deck[hands[player-1]] -= 1
#     print hands
    for i in xrange(len(key[8:])):
        if key[8 + i] == 'k':
            hands[i] = 13
            deck[hands[i]] -= 1
            if deck[hands[i]] < 0:
                return -INF #impossible key
    cumrel = 0
    equity = 0
    for rel in generate_hands(hands):        
        if rel <= 0:
            continue
        if not allowed(hands,key):
#             print 'not allowed', hands,key
            continue
#         print "allowed", hands,key,equity,rel,cumrel
        cumrel += rel
        equity += rel*simulate(hands,key)
    if cumrel > 0:
        return float(equity) / cumrel
    else:
        return -INF
    
#     else: #if situation is not possible, estimate the equity
#         cumrel = 0
#         equity = 0
#         for rel in generate_hands(hands):
#             if rel <= 0:
#                 continue
#             cumrel += rel
#             equity += rel*get_score(hands)[player]
#         if cumrel > 0:
#             return float(equity) / cumrel
#         else:
#             return -INF

In [14]:
import random
import sys
import time
def Solve(players):
    global policy
    global passing
    global staying
    iterations = 100
    INF = float('inf')
    try:
        passing.keys()
    except:
        passing = {}
    try:
        staying.keys()
    except:
        staying = {}
    try:
        policy.keys()
    except:
        policy = {}
        
    t0 = time.clock()
    MOVEMENT = 1
    print 'Now solving {} players'.format(players)
    keys = 0
    for key in key_gen(players):
        if not key in frozen:
#             passing[key] = 0
#             staying[key] = 1
            passing[key] = random.random()
            staying[key] = random.random()
        policy[key] = 'e' if passing[key] == staying[key] else 'p' if passing[key] > staying[key] else 's'
        keys += 1

    for i in xrange(iterations):
        old_passing = copy(passing)
        old_staying = copy(staying)
        old_policy = copy(policy)
        key_count = 0
        for key in key_gen(players):
            if key in frozen:
                continue
            progress(float(key_count)/keys,size = 50)
#             sys.stdout.write('{:2.2f}% complete\r'.format((100.*key_count)/keys))
            sys.stdout.flush()
            key_count += 1

            #passing
#             passing[key] = INF
#             staying[key] = -INF
            policy[key] = 'p'

            passing_equity = equity(key)
            #staying
#             passing[key] = -INF
#             staying[key] = INF
            policy[key] = 's'
            staying_equity = equity(key)

            if passing_equity == -INF:
                passing[key] = old_passing[key]
            else:
                passing[key] = (1-MOVEMENT)*old_passing[key] + MOVEMENT * passing_equity

            if staying_equity == -INF:
                staying[key] = old_staying[key]
            else:
                staying[key] = (1-MOVEMENT)*old_staying[key] + MOVEMENT * staying_equity


            policy[key] = 'e' if passing[key] == staying[key] else 'p' if passing[key] > staying[key] else 's'
            if passing_equity == staying_equity == -INF:
                policy[key] = old_policy[key][0] + 'n'
#                 policy[key] = 'sn' if random.random()>0.5 else 'pn'

            if passing_equity == -INF and staying_equity > -INF:
                assert(staying[key] > passing[key])
            if staying_equity == -INF and passing_equity > -INF:
                assert(passing[key] > staying[key])

        distance = max([abs(old_passing[key]-passing[key]) for key in key_gen(players)] + 
                       [abs(old_staying[key]-staying[key]) for key in key_gen(players)])
        ham = sum([policy[key] != old_policy[key] for key in policy])
        sys.stdout.write(' '*100+'\r')
        print "largest jump: {:1.2f}, policy changes: {:4d}".format(distance,ham)   

#         print distance
        sys.stdout.flush()
        time.sleep(0.0001)
        if ham == 0:
            print 'done. Time elapsed: {:8.2f}s'.format(time.clock()-t0)
            break
#         for key in policy:
#             if key[2:4] != '00':
#                 continue
#             if policy[key] != old_policy[key]:
#                 print key,policy[key],old_policy[key]
#         if distance < 0.00001:
#             MOVEMENT = 1

#     print sum([policy5[key] != policy[key] for key in key_gen(players) if policy[key][-1] != 'n' and policy5[key][-1] != 'n'])
    sys.stdout.flush()

def get_profile():
    return profile

In [11]:
Solve(3)

Now solving 3 players
largest jump: 1.66, policy changes:  411                                                            
largest jump: 1.65, policy changes:   79                                                            
largest jump: 0.39, policy changes:   79                                                            
largest jump: 0.21, policy changes:   41                                                            
largest jump: 0.05, policy changes:    0                                                            
done. Time elapsed:     5.19s


In [8]:
equity('02010800s')

-0.28

In [15]:
import cProfile
cProfile.run('Solve(4)',sort = 'tottime')

Now solving 4 players
largest jump: 1.74, policy changes: 1005                                                            
largest jump: 1.68, policy changes:  125                                                            
largest jump: 0.30, policy changes:   98                                                            
largest jump: 0.16, policy changes:   13                                                            
largest jump: 0.03, policy changes:    0                                                            
done. Time elapsed:    88.74s
         93022596 function calls (77704436 primitive calls) in 88.745 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  4238520   33.801    0.000   48.087    0.000 <ipython-input-13-3f8899ff2d03>:89(allowed)
19570720/4252560   12.611    0.000   13.774    0.000 <ipython-input-13-3f8899ff2d03>:24(generate_hands)
  7771009   10.056    0.000   10.056    0.000 {method 'format' of 's

In [819]:
with open('solution.txt','w') as file_:
    count = 0
    for key in passing:
        file_.write("{} : {} , {} , {}\n".format(key , policy[key],passing[key],staying[key]))

In [820]:
saved_policy = copy(policy)
saved_passing = copy(passing)
saved_staying = copy(staying)

In [9]:
general_policy = {}
players = 2
for key in key_gen(players):
    smallkey = key[:6]
    if policy[key][-1] == 'n':
        continue
    if int(key[6:8]) and int(key[6:8]) < int(key[4:6]):
        continue
    if not smallkey in general_policy:
        general_policy[smallkey] = [-INF,INF]
    
    ps = key.count('ps') + key.count('bk')
    if policy[key] == 'p':
        general_policy[smallkey][0] = max(general_policy[smallkey][0],ps)
    if policy[key] == 's':
        general_policy[smallkey][1] = min(general_policy[smallkey][1],ps)
    
conflicted = set()
for key in general_policy:
    if general_policy[key][0]== -INF and general_policy[key][1] != INF:
        print key , 'stay'
    elif general_policy[key][0]!= -INF and general_policy[key][1] == INF:
        print key , 'pass'        
    elif general_policy[key][1] - general_policy[key][0] == 1:
        print key , 'stay on {}'.format(general_policy[key][1])
    else:
        conflicted.add(key)
        print key , general_policy[key]
print '\n###################################################\n'
for key in conflicted:
    print 'conflicted key: ',key, general_policy[key]
    ps_bin = [[] for _ in xrange(3)]
    for bigkey in key_gen(players):
        if policy[bigkey][-1] == 'n':
            continue
        if int(bigkey[6:8]) and int(bigkey[6:8]) < int(bigkey[4:6]):
            continue
        ps = bigkey.count('ps') + bigkey.count('bk')
        if bigkey[:6] == key:
            ps_bin[ps].append((bigkey,ps,policy[bigkey],passing[bigkey],staying[bigkey]))
    for i in xrange(general_policy[key][1],general_policy[key][0]+1):
        for entry in ps_bin[i]:
            print ', '.join(map(str,entry))
    print 

020013 stay
020012 stay
020011 stay
020010 stay
020112 stay
020113 stay
020110 stay
020111 stay
020108 pass
020104 pass
020004 pass
020005 pass
020006 pass
020007 pass
020109 stay
020001 pass
020002 pass
020003 pass
020105 pass
020008 stay
020107 pass
020106 pass
020101 pass
020009 stay
020103 pass
020102 pass

###################################################



## Making Spreadsheets

In [20]:
import pandas as pd
from copy import copy

def get_policy(smallkey):
    players = int(smallkey[:2])
    for bigkey in key_gen(players):
        if policy[bigkey][-1] == 'n' or int(bigkey[6:8]) and int(bigkey[6:8]) < int(bigkey[4:6]):
            continue  
        if bigkey[:6] == smallkey:
            print bigkey,policy[bigkey]

In [21]:
# writer = pd.ExcelWriter('solution_excel.xlsx',engine = 'xlsxwriter')
for players in xrange(4,5):
    general_policy = {}
    policy_dict={}
    full_dict = {}
    default_dict = {i:'' for i in xrange(1,14)}

    #join the keys into a 2-axis system
    for key in key_gen(players):

        if (policy[key][-1] == 'n' or
            int(key[6:8]) and int(key[6:8]) < int(key[4:6])):
            continue        
        player = int(key[2:4])+1
        card = int(key[4:6])
        smallkey = key[:6]
        if not player in full_dict:
            full_dict[player] = copy(default_dict)
        if not smallkey in general_policy:
            general_policy[smallkey] = {'p':[0 for i in xrange((player+1)/2)], 
                                        's':[0 for i in xrange((player+1)/2)],
                                        'e':''}
        full_dict[player][card] += (key[8:] + 
                                    (key[6:8] if key[-1] == 'p' else '') + 
                                    ': ' + 
                                    ('pass' if policy[key] == 'p' else
                                     'stay' if policy[key] == 's' else
                                     'either') + '\n')
        ps = key.count('ps') + key.count('bk')
        if policy[key] == 'p':
            general_policy[smallkey]['p'][ps] += 1
        if policy[key] == 's':
            general_policy[smallkey]['s'][ps] += 1
        if policy[key] == 'e':
            general_policy[smallkey]['p'][ps] += 1
            general_policy[smallkey]['s'][ps] += 1
            general_policy[smallkey]['e'] = '~'
            

    for key in general_policy:
        player = int(key[2:4])+1
        card = int(key[4:6])
        if not player in policy_dict:
            policy_dict[player] = copy(default_dict)
        if sum(general_policy[key]['s']) == 0:
            policy_dict[player][card] = 'pass'
        elif sum(general_policy[key]['p']) == 0:
            policy_dict[player][card] = 'stay'
        else:
            i = 0
            policy_dict[player][card] = ''
            while i <= (player-1)/2 and general_policy[key]['s'][i]==0:
                i+=1
            if i>0:
                policy_dict[player][card] += 'p{}'.format(i-1)
            while i <= (player-1)/2 and sum(general_policy[key]['p'][i:]):
                policy_dict[player][card] += '/' if policy_dict[player][card] else ''
                if general_policy[key]['p'][i] > general_policy[key]['s'][i]:
                    if general_policy[key]['s'][i]:
                        policy_dict[player][card] += 'p*{}'.format(i)
                    else:
                        policy_dict[player][card] += 'p{}'.format(i)
                elif general_policy[key]['p'][i] < general_policy[key]['s'][i]:
                    if general_policy[key]['p'][i]:
                        policy_dict[player][card] += 's*{}'.format(i)
                    else:
                        policy_dict[player][card] += 's{}'.format(i)
                else:
                    policy_dict[player][card] += '(p=s)*{}'.format(i)
                i += 1
            if i <= (player-1)/2:
                policy_dict[player][card] += '/s{}'.format(i)
            
#         if general_policy[key][0]== -INF and general_policy[key][1] != INF:
#             policy_dict[player][card] = 'stay'+general_policy[key][2]
#         elif general_policy[key][0]!= -INF and general_policy[key][1] == INF:
#             policy_dict[player][card] = 'pass'+general_policy[key][2]
#         elif general_policy[key][1] - general_policy[key][0] == 1:
#             policy_dict[player][card] = 'p{}/s{}'.format(
#                 general_policy[key][0],general_policy[key][1])+general_policy[key][2]
#         elif general_policy[key][1] <= general_policy[key][0]:
#             policy_dict[player][card] = ''
#             if general_policy[key][1] > 0:
#                 policy_dict[player][card] += 'p{}'.format(general_policy[key][1]-1)
#             for i in range(general_policy[key][1],general_policy[key][0]+1):
#                 policy_dict[player][card] += '/' if len(policy_dict[player][card])>0 else ''
#                 passes = 0
#                 stays = 0
#                 for bigkey in key_gen(players):
#                     if (policy[bigkey][-1] == 'n' or
#                         (int(bigkey[6:8]) and int(bigkey[6:8]) < int(bigkey[4:6])) or 
#                         bigkey[:6] != key):
#                         continue
#                     ps = bigkey.count('ps') + bigkey.count('bk')
#                     if ps==i:
#                         if policy[bigkey] == 'p':
#                             passes += 1
#                         if policy[bigkey] == 's':
#                             stays += 1
#                 if passes > stays:
#                     policy_dict[player][card] += 'p*{}'.format(i,)
#                 if stays > passes:
#                     policy_dict[player][card] += 's*{}'.format(i)
#                 if stays == passes:
#                     print stays
#                     print passes
#                     policy_dict[player][card] += '(p=s)*{}'.format(i)
#             if general_policy[key][0] < (player-1)/2:
#                 policy_dict[player][card] += '/s{}'.format(general_policy[key][0]+1)
#         else:
#             policy_dict[player][card] = '???'

    policy_df = pd.DataFrame.from_dict(policy_dict,orient = 'index')        
    policy_df.index.name = 'player'
    policy_df.columns.name = 'card'
    full_df = pd.DataFrame.from_dict(full_dict,orient = 'index')        
    full_df.index.name = 'player'
    full_df.columns.name = 'card'
#     policy_df.to_excel(writer,sheet_name = '{} players policy'.format(players))
#     full_df.to_excel(writer,sheet_name = '{} players verbose'.format(players))
    from IPython.display import display, HTML
    display(HTML(policy_df.to_html().replace("\\n","<br>")))
    # display(HTML(full_df.to_html().replace("\\n","<br>")))
# writer.save()

card,1,2,3,4,5,6,7,8,9,10,11,12,13
player,,,,,,,,,,,,,
1,pass,pass,pass,pass,pass,stay,stay,stay,stay,stay,stay,stay,stay
2,pass,pass,pass,pass,pass,pass,stay,stay,stay,stay,stay,stay,stay
3,pass,pass,pass,p0/(p=s)*1,p0/s1,p0/s1,p0/s1,stay,stay,stay,stay,stay,stay
4,pass,pass,pass,p0/p*1,p0/s*1,p0/s1,p0/s1,stay,stay,stay,stay,stay,stay


In [968]:
get_policy('030206')

03020600ss p
03020606sp p
03020607sp p
03020608sp p
03020609sp p
03020610sp p
03020611sp p
03020612sp p
03020600ps s
03020606pp p
03020607pp p
03020608pp p
03020609pp p
03020610pp p
03020611pp p
03020612pp p
03020600bk s


In [1012]:
policy5 = {}
for key in key_gen(5):
    policy5[key] = policy[key]

In [996]:
for key in key_gen(3):
    if policy2[key] != policy[key] and policy2[key][0] != 'n' and policy[key][0] != 'n':
        print key,policy[key],policy2[key]

KeyError: '03000100'

In [14]:
policy

{'04030102spp': 'sn',
 '04031311spp': 'pn',
 '03020500bk': 'sn',
 '03020804pp': 'pn',
 '04030503bkp': 'sn',
 '04021306sp': 'sn',
 '03020901sp': 'pn',
 '03020708pp': 'sn',
 '03010405p': 'pn',
 '04020606sp': 'pn',
 '04020901pp': 'sn',
 '03011107p': 'pn',
 '04020908pp': 'pn',
 '03010505p': 'sn',
 '03011007p': 'pn',
 '03021203sp': 'pn',
 '03020701sp': 'pn',
 '04030501ppp': 'pn',
 '03020403pp': 'pn',
 '04030203ppp': 'pn',
 '04021107pp': 'pn',
 '03020500ps': 'pn',
 '04011007p': 'pn',
 '04020512pp': 'pn',
 '04010102p': 'sn',
 '03011110p': 'sn',
 '04010505p': 'pn',
 '04011212p': 'pn',
 '04031011ssp': 'sn',
 '04020211pp': 'sn',
 '04030708psp': 'sn',
 '04030206psp': 'pn',
 '04021201sp': 'sn',
 '03020902sp': 'sn',
 '04030100pss': 'sn',
 '03010412p': 'sn',
 '04010608p': 'pn',
 '04020811pp': 'pn',
 '04021308sp': 'pn',
 '04031012psp': 'pn',
 '04030509ssp': 'sn',
 '04030602psp': 'sn',
 '04030305ppp': 'sn',
 '04030107bkp': 'sn',
 '04030108spp': 'pn',
 '03021303pp': 'pn',
 '04030504psp': 'sn',
 '040206

In [76]:
for key in key_gen(2):
    print key,policy[key],passing[key],staying[key]

02000100 p -0.0737254901961 -0.482352941176
02000200 p -0.0698039215686 -0.435294117647
02000300 p -0.0721568627451 -0.388235294118
02000400 p -0.0807843137255 -0.341176470588
02000500 p -0.0956862745098 -0.294117647059
02000600 p -0.116862745098 -0.247058823529
02000700 p -0.14431372549 -0.2
02000800 s -0.182745098039 -0.152941176471
02000900 s -0.0721568627451 -0.0627450980392
02001000 s -0.0737254901961 0.0658823529412
02001100 s -0.185882352941 0.194509803922
02001200 s -0.249411764706 0.323137254902
02001300 s -0.196078431373 0.451764705882
02010100s p -0.27 -0.5
02010101p p 0.48 0.0
02010102p p 0.41 -0.5
02010103p p 0.33 -0.5
02010104p p 0.25 -0.5
02010105p p 0.17 -0.5
02010106p p 0.09 -0.5
02010107p p 0.01 -0.5
02010108p p -0.07 -0.5
02010109p p -0.15 -0.5
02010110p p -0.23 -0.5
02010111p p -0.31 -0.5
02010112p p -0.39 -0.5
02010200s p -0.27 -0.5
02010201p s 0.47 0.5
02010202p p 0.4 0.0
02010203p p 0.33 -0.5
02010204p p 0.25 -0.5
02010205p p 0.17 -0.5
02010206p p 0.09 -0.5
02010